Импортируем необходимые модули

In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
#Jupyter Notebook Rdnderers

Задаём функцию $z=f(x,y)=e^{-\frac{x^2+y^2}{1}}-e^{-\frac{(x-0.75)^2+(y-0.75)^2}{1/2}}$

In [2]:
def f(x,y):
    return np.exp(-(x**2+y**2))-np.exp(-((x-0.75)**2+(y-0.75)**2)/(1/2))

Введём сетку и вычислим значения функции в её узлах 

$\{x_i\}=\{x_0, x_1, \dots, x_{99}\}$, с шагом $h_x\approx0.01$ 

$\{y_j\}=\{x_0, x_1, \dots, x_{99}\}$, с шагом $h_y\approx0.01$ 

$z_{i,j}=f(x_i,y_j)$ 

In [3]:
x=np.linspace(0,1,100)
y=np.linspace(0,1,100)
z=np.zeros((len(x),len(y)))

hx=x[1]-x[0]
hy=y[1]-y[0]


for i,x_item in enumerate(x):
    for j,y_item in enumerate(y):
        z[i,j]=f(x_item,y_item)

Строим график функции

In [4]:
fig =  make_subplots(rows=1, cols=1 ,print_grid=False)
fig.add_trace(go.Heatmap(x=x,y=y,z=z.T,colorscale='jet',zsmooth = 'best'))
fig.update_layout(height=600,width=600)
fig.show()


Градиент и производная по направлению

$\nabla f(x,y)=\frac{\partial f}{\partial x}\vec{i}+\frac{\partial f}{\partial y}\vec{j}$,

$\nabla_{\vec{i}} f(x,y)=\left(\nabla f(x,y),\vec{i}\right)$,

$\nabla_{\vec{j}} f(x,y)=\left(\nabla f(x,y),\vec{j}\right)$.

In [5]:
grad_z=np.gradient(z,hx,hy)
dzdi=grad_z[0]
dzdj=grad_z[1]


In [6]:
fig =  make_subplots(rows=2, cols=2 ,print_grid=False,
                    subplot_titles=('original function', 'maximum value of the derivative',
                    'in the direction of the Ox axis',
                    'in the direction of the Oy axis'
                   ))
fig.add_trace(go.Heatmap(
            z=z.T,
            colorscale='jet',x=x,y=y,zmid=0,
            zsmooth = 'best'),row=1,col=1)
fig.add_trace(go.Heatmap(
            z=((dzdi**2+dzdj**2)**0.5).T,
            colorscale='jet',x=x,y=y,
            zsmooth = 'best'),row=1,col=2)
fig.add_trace(go.Heatmap(
            z=dzdi.T,
            colorscale='jet',x=x,y=y,zmid=0,
            zsmooth = 'best'),row=2,col=1)
fig.add_trace(go.Heatmap(
            z=dzdj.T,
            colorscale='jet',x=x,y=y,zmid=0,
            zsmooth = 'best'),row=2,col=2)


fig.update_layout(height=900,width=900)
fig.update_xaxes(title='x')
fig.update_yaxes(title='y')
fig.update_traces(showscale=False)
fig.show()

Для наглядности строим трёхмерный график

In [7]:
fig = make_subplots(rows=2, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}]],
    subplot_titles=('original function', 'maximum value of the derivative',
                    'in the direction of the Ox axis',
                    'in the direction of the Oy axis'))

fig.add_trace(
    go.Surface(x=x, y=y, z=z.T, colorscale='jet', showscale=False),
    row=1, col=1)
    
fig.add_trace(
    go.Surface(x=x, y=y, z=((dzdi**2+dzdj**2)**0.5).T, colorscale='jet', showscale=False),
    row=1, col=2)
    
fig.add_trace(
    go.Surface(x=x, y=y, z=dzdi.T, colorscale='jet', showscale=False),
    row=2, col=1)
    
fig.add_trace(
    go.Surface(x=x, y=y, z=dzdj.T, colorscale='jet', showscale=False),
    row=2, col=2)
    
fig.update_layout(height=900,width=900)

fig.show()

Метод градиентного спуска для поиска минимума функции

Идея: двигаться в направлении антиградиента.

$\vec{\xi}^{[0]}=\begin{pmatrix} x_{нач}\\y_{нач} \end{pmatrix},$ 

$\vec{\xi}^{[n+1]}=\vec{\xi}^{[n]}-\mu\nabla f\left(\vec{\xi}^{[n]}\right).$ 


Определим функцию одного шага

In [8]:
def step(x,y,mu):

    i=int(x/hx)
    j=int(y/hy)

    new_x=x-mu*dzdi[i,j]
    new_y=y-mu*dzdj[i,j]
    
    return new_x,new_y

Определим функцию $N$ шагов из точки $\vec{\xi}^{[0]}$ с постоянной скоростью спуска $\mu$

In [9]:
def optim(N,ksi,mu):
    mass_x=[ksi[0]]
    mass_y=[ksi[1]]
    for i in range(N):
        new_x,new_y=step(mass_x[-1],mass_y[-1],mu)
        mass_x.append(new_x)
        mass_y.append(new_y)
    return mass_x,mass_y

Зададим параметры и вычислим три траектории из различных начальных точек

In [10]:
mu=0.01
N=500
ksi_1=[0,0]
ksi_2=[0,0.75]
ksi_3=[0.75,0]

track_1_x,track_1_y=optim(N,ksi_1,mu)
track_2_x,track_2_y=optim(N,ksi_2,mu)
track_3_x,track_3_y=optim(N,ksi_3,mu)


Строим график

In [11]:
fig =  make_subplots(rows=1, cols=1 ,print_grid=False)

fig.add_trace(go.Heatmap(x=x,y=y,z=z,colorscale='jet',zsmooth = 'best'),row=1,col=1)

fig.add_trace(go.Scatter(x=track_1_x, y=track_1_y,line=dict(color='black', width=2),mode='lines+markers') ,row=1, col=1)
fig.add_trace(go.Scatter(x=track_2_x, y=track_2_y,line=dict(color='black', width=2),mode='lines+markers') ,row=1, col=1)
fig.add_trace(go.Scatter(x=track_3_x, y=track_3_y,line=dict(color='black', width=2),mode='lines+markers') ,row=1, col=1)

fig.update_layout(height=600,width=600,showlegend=False,title='Gradient descent trajectories')

fig.update_xaxes(range=[0,1])
fig.update_yaxes(range=[0,1])

fig.show()

Определим вспомогательную функцию для вычисления массива значений функции $z=f(x,y)$ в точках траекторий

In [12]:
def get_z(x,y):
    z=[]
    for i in range(len(x)):
        x_item=x[i]
        y_item=y[i]
        z_item=f(x_item,y_item)
        z.append(z_item)
    return z

In [13]:
track_1_z=get_z(track_1_x,track_1_y)
track_2_z=get_z(track_2_x,track_2_y)
track_3_z=get_z(track_3_x,track_3_y)


Для наглядности строим трёхмерный график

In [14]:
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y,colorscale='jet',opacity=0.5)])

fig.add_traces(go.Scatter3d(x=track_1_x, y=track_1_y, z=track_1_z,line=dict(color='red', width=5),mode='lines+markers'))
fig.add_traces(go.Scatter3d(x=track_2_x, y=track_2_y, z=track_2_z,line=dict(color='green', width=5),mode='lines+markers'))
fig.add_traces(go.Scatter3d(x=track_3_x, y=track_3_y, z=track_3_z,line=dict(color='blue', width=5),mode='lines+markers'))

fig.update_layout(title='Gradient descent', autosize=False,showlegend=False,width=900, height=900)

fig.show()